 # **Ejercicio 3**
 ## **Modelos de Clasificación**. 
Considere el conjunto de datos `NeurIPS 2024- Predict New Medicines with BELKA`. Implemente la versión de clasificación para cada uno de los modelos estudiados en clases, esto es Regresión Logística y KNN. Construir una tabla de error que contenga las métricas usuales de clasificación: precission, recall, f1-score, AUC. Además, agregue matrices de confusión (ver confusion_matrix) y curvas ROC (ver plot_roc). Puede utilizar la librería GridSearchCV y Pipeline para evaluar cada modelo. Verifique que la validación cruzada seleccionada es la adecuada, y justifíquelo. Utilice la métrica AUC, para seleccionar el mejor modelo de clasificación (maximizar AUC). Los resultados deben estar registrados en una tabla de error (ver Tabla 1) que resuma cada score obtenido por modelo implementado.

\[
\begin{array}{|c|c|c|c|c|}
\hline
\textbf{Modelo} & \textbf{precision} & \textbf{recall} & \textbf{f1-score} & \textbf{AUC} \\
\hline
\text{K-NN} & \cdots & \cdots & \cdots & \cdots \\
\text{Logistic Regression} & \cdots & \cdots & \cdots & \cdots \\
\hline
\end{array}
\]

\text{Cuadro 1: Modelo de clasificación para detección de fraude.}
